In [1]:
import numpy as np
import networkx as nx
import pandas as pd
from twittergraph import TwitterGraph as tg
from sklearn.cluster import KMeans
from collections import defaultdict
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
graph = tg.rt_graph_from_json('/Volumes/pond/Temp/twitter/', 0)

In [3]:
counter = 0
for u, v in graph.edges_iter():
    counter += 1

print counter

58193


In [8]:
# just a pile of fun hashtag related stuff
# some things don't need to be run again so be careful!

In [53]:
hts = sorted(tg.ht_counts.items(), key=lambda x: x[1])

In [11]:
for ht in hts:
    if ('eff' in ht[0] or 'malema' in ht[0] or 'julius' in ht[0]) and ht[1] > 10:
        print("%s: %d" % (ht[0], ht[1]))
print("\n\n")

for ht in hts:
    if('anc' in ht[0] or 'jacob' in ht[0] or 'zuma' in ht[0]) and ht[1] > 10:
        print("%s: %d" %(ht[0], ht[1]))
print("\n\n")

for ht in hts:
    if('da' in ht[0] or 'zille' in ht[0] or 'hellen' in ht[0]) and ht[1] > 10:
        print("%s: %d" % (ht[0], ht[1]))

freestateeff: 11
juliusmalema: 14
effcocktailparty: 14
ancvseff: 15
effcandidatelist: 15
effwelkom: 16
effkliptown: 16
effmarchsabc: 16
effkliptownrally: 21
effkhayelitsha: 27
effmanifesto: 32
effmafikeng: 34
effmdantsane: 37
effighter: 37
effthankyou: 38
effrally: 143
malema: 198
malemaqanda: 239
voteeff: 790
eff: 3952



ancgaladinner: 11
zumaterm2: 11
anclives: 12
ancnkandla: 12
democraticalliance: 13
ancleads: 14
ancvseff: 15
byedahelloanc: 15
ancayisafani: 17
dancmanifesto: 17
ancvictoryparty: 17
zumasanc: 18
ancinthestreests: 19
zumainauguration: 19
voteanc7may: 19
ancendorsement: 20
ancyl: 21
stopzuma: 22
carteblanche: 22
massvoteanc: 27
vivaanc: 28
ancfraud: 31
knowyouranc: 32
zumaout: 33
whyivoteanc: 35
jacobzuma: 41
ancfail: 41
ancvictory: 44
proudlybroughtbyanc: 49
zanc: 51
zumaneverlovedus: 53
firingzumaanc7thmay: 57
zumamustgo: 58
thankyouanc: 62
ancbreakfast: 70
ivoteanc: 82
ancwins: 83
sanc: 85
iamanc: 106
ancsiyanqoba: 140
ancrally: 140
myanc: 140
zillevszuma: 146
ancin

In [19]:
print graph.degree('eff')

1789


In [7]:
graph_copy = graph.copy()
for node in graph.nodes_iter():
    if graph.node[node]['type'] == 'hashtag':
        graph_copy.remove_node(node)

In [8]:
nx.write_adjlist(graph_copy, 'SA.adjlist')

In [9]:
dw = pd.read_csv('SA.walked', sep=" ", skiprows=1, header=None)

In [12]:
kmean = KMeans(n_clusters=3)
kmean.fit(dw.iloc[:, 1:65])

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances=True, random_state=None, tol=0.0001,
    verbose=0)

In [ ]:
dw['label'] = kmean.labels_
cluster_assn = pd.DataFrame(columns=["node", "cluster"])
cluster_assn['node'] = dw.iloc[:, 0]
cluster_assn['cluster'] = dw.iloc[:, 65]
cluster_assn.to_csv('SA_DW_Clusters.csv')

In [8]:
cluster_df = pd.read_csv('SA_DW_Clusters.csv')
cluster_dict = {}
for i in range(0,cluster_df.shape[0]):
    cluster_dict[cluster_df.loc[i, 'node']] = cluster_df.loc[i, 'cluster']

In [21]:
count = 0
for node in graph.nodes_iter():
    print count
    if graph.node[node]['type'] == 'hashtag':
        continue
    if count > 1:
        break

    for u, v in graph.edges(node):
        if graph.node[v]['type'] != 'hashtag':
            for t_id in graph.edge[u][v]['tweets']:
                count += 1
                # print(tg.tweets[t_id])
                # print("\n")

0
230


In [52]:
ht_info = [defaultdict(int), defaultdict(int), defaultdict(int)]
stop_tags = ['elections2014', 'da', 'eff', 'anc', 'siyanqoba', 'ayisafani', 'southafrica', 'liar', 'zuma', 'nkandla', 'africa', 'vote', 'sabcnews']
sia = SentimentIntensityAnalyzer()
for node in graph.nodes_iter():
    if graph.node[node]['type'] == 'hashtag':
        continue
    cluster = cluster_dict[node]
    for u, v in graph.edges(node):
        if graph.node[v]['type'] == 'hashtag' and v not in stop_tags:
            for tweet in graph.edge[u][v]['tweets']:
                vs = sia.polarity_scores(tg.tweets[tweet])
                if vs['pos'] > .25:
                    ht_info[cluster][v] += 1

In [52]:
for i in range(0, 3):
    hts = reversed(sorted(ht_info[i].items(), key=lambda x: x[1]))
    count = 0
    for ht in hts:
        print("%s - %d" % (ht[0], ht[1]))
        count += 1
        if count == 5:
            break
    print("\n\n")

voteeff - 86
asijiki - 61
voteanc - 42
malemaqanda - 26
sikbhacile - 26



ivoteda - 174
daresults - 140
voteforjobs - 139
teamda - 114
voteda - 104



iecmustanswer - 41
dmgathering - 20
ivoteda - 17
nkandlareport - 16
saelections2014 - 12





In [ ]:
"""
before analysis:
elections2014 - 1846
eff - 1212
siyanqoba - 961
anc - 700
ayisafani - 511



elections2014 - 1866
ayisafani - 1832
da - 1091
anc - 754
ivoteda - 693



elections2014 - 2188
siyanqoba - 606
ayisafani - 505
anc - 463
da - 439
"""